In [1]:
import os
import sys

In [2]:
import pandas as pd
# Import data
df_raw = pd.read_excel('../Data/DATA_UFM_combined.xlsx', header=0)

# Columns to keep
cols = [
    'udbud_id',
    'titel',
    'educational_category',
    'displaydocclass',
    'hovedinsttx',
    'instregiontx',
    'instkommunetx',
    'arbejdstid_timer',
    'arbmedstud_likert',
    'ensom_likert',
    'maanedloen_10aar',
    'maanedloen_nyudd',
    'tidsforbrug_p50',
    'stress_daglig_likert'
]

data = df_raw[cols]

# Remove all udbud_id==999999, as this is the education on national level
data_whole_edu = data[data['udbud_id'] == 999999]
data = data[data['udbud_id'] != 999999]

# Remove the udbud_id column
data = data.drop(columns=['udbud_id'])

data_na = data.copy()
# Remove all rows with missing values
data = data.dropna()

In [3]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool

output_notebook()

# Create figure
p = figure(
    width=700, 
    height=500,
    title="Stress vs Loneliness by Educational Program",
    x_axis_label="Daily Stress (Likert)",
    y_axis_label="Loneliness (Likert)"
)

# Add scatter plot
p.circle(
    x='stress_daglig_likert',
    y='ensom_likert',
    size=8,
    alpha=0.5,
    source=data_whole_edu
)

# Add hover tool to show program title
hover = HoverTool(tooltips=[
    ("Program", "@titel"),
    ("Stress", "@stress_daglig_likert{0.00}"),
    ("Loneliness", "@ensom_likert{0.00}")
])
p.add_tools(hover)

show(p)

Loading BokehJS ...

In [6]:
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, DataTable, TableColumn, HTMLTemplateFormatter, NumberFormatter

output_notebook()

num_cols = [
    "arbejdstid_timer",
    "maanedloen_nyudd",
    "maanedloen_10aar",
    "tidsforbrug_p50",
    "stress_daglig_likert",
    "ensom_likert",
    "arbmedstud_likert",
]
cat_cols = [
    "titel",
    "educational_category",
    "displaydocclass",
    "hovedinsttx",
    "instregiontx",
    "instkommunetx",
]

df_norm = data[num_cols + cat_cols].copy()

# normalize numeric columns to 0–100 for bar widths
for c in num_cols:
    lo, hi = df_norm[c].min(), df_norm[c].max()
    df_norm[c + "_norm"] = 0 if hi == lo else (df_norm[c] - lo) / (hi - lo) * 100

source = ColumnDataSource(df_norm)

# bar cell templates (different colors per column if you want)
tpl = lambda rgb: f"""
<div style="height:14px;border-radius:2px;background:
linear-gradient(to right, rgba({rgb},0.25), rgba({rgb},0.95));
width:<%= value %>%"></div>"""

palette = {
    "arbejdstid_timer_norm":     "30,136,229",  # blue
    "maanedloen_nyudd_norm":     "240,98,146",  # pink
    "maanedloen_10aar_norm":     "229,57,53",   # red
    "tidsforbrug_p50_norm":      "67,160,71",   # green
    "stress_daglig_likert_norm": "245,124,0",   # orange
    "ensom_likert_norm":         "123,31,162",  # purple
    "arbmedstud_likert_norm":    "0,121,107",   # teal
}

columns = []
# numeric columns as bars
for norm_col, rgb in palette.items():
    title = norm_col.replace("_norm","").replace("_"," ").title()
    columns.append(
        TableColumn(field=norm_col, title=title,
                    formatter=HTMLTemplateFormatter(template=tpl(rgb)))
    )
# categorical text columns
for c in cat_cols:
    columns.append(TableColumn(field=c, title=c.replace("_"," ").title()))

table = DataTable(
    source=source,
    columns=columns,
    width=1100,
    height=520,
    index_position=None,
    reorderable=False,
    sortable=True,
)

show(table)


Loading BokehJS ...